## Setup

In [ ]:
!pip install langchain ctransformers

In [ ]:
from langchain.llms import CTransformers
from langchain import PromptTemplate

llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML',
                    model_file='llama-2-7b-chat.ggmlv3.q4_0.bin',
                    model_type="llama",
                    gpu_layers=5, context_length=8196, reset=True, threads=8,
                    top_k=20, top_p=0.95, temperature=0)

**Smoke test**: Check that the model is loaded successfully and generates good-quality responses

In [ ]:
template = """
### Instruction: {query}. Be succinct, and return response as a 5-point list.
### Response:"""
prompt_template = PromptTemplate(input_variables=["query"],
                        template=template)
prompt = prompt_template.format(query="How can I be a more balanced human being?")
print(llm(prompt))

# Note: this cell should take about ~30-60 seconds to run successfully (on local)
# Note: this cell should take about ~60-80 seconds to run successfully (on an instance with GPUs)
# Note: this cell should take about ~150-170 seconds to run successfully (on an instance with CPUs)

In [ ]:
# common variables that we'll use in multiple exercises

RESUME = """
Objective: Dedicated IT Developer with over 5 years of experience in full-stack web development, mobile application development, and cloud computing. Seeking to leverage my technical expertise and problem-solving skills to contribute to a forward-thinking team at WidgetCraft.

Technical Skills:
- Languages: Java, Python, JavaScript, C#
- Web: HTML5, CSS3, Bootstrap, React, Angular, Node.js
- Mobile: Android (Java, Kotlin), iOS (Swift)
"""

## Exercise 1: Manual exploratory testing

In [ ]:
template = """
### Instruction: Extract technical skills from this document:
{resume}
### Response:"""

prompt_template = PromptTemplate(input_variables=["resume"], template=template)
prompt = prompt_template.format(resume=RESUME)

print(llm(prompt))

The result is human-readable, but not a very helpful data structure. Let's get the model to return JSON so we can work with it in automated tests (Exercise 2) and any other downstream components.

In [ ]:
template = """
### Instruction: Extract technical skills from this document using only information in the following document. Return results in valid JSON format.
{resume}
### Response:
"""

prompt_template = PromptTemplate(input_variables=["resume"], template=template)
prompt = prompt_template.format(resume=RESUME)

result = llm(prompt)
print(result)

## Exercise 2: Automated tests. Example-based tests

In [ ]:
import json

template = """
### Instruction: Extract technical skills from this document using only information in the following document. Return results in valid JSON format.
Final answer should be in the following format:

{{"technical_skills": {{"key_1": ["value1", "value2", "value3" ], "key_2": ["value1", "value2", "value3"]}}}}

Ensure that key_1, key_2, etc. are exact valid keys from user input, presented in snake_case

### Input: {resume}

### Response:"""
prompt_template = PromptTemplate(
    input_variables=["resume"],
    template=template,
)
prompt = prompt_template.format(resume=RESUME)
result = llm(prompt)

print(result)

In [ ]:
actual_skills = json.loads(result)

expected_skills = {
    "technical_skills": {
        "languages": ["Java", "Python", "JavaScript", "C#"],
        "web": ["HTML5", "CSS3", "Bootstrap", "React", "Angular", "Node.js"],
        "mobile": ["Android (Java, Kotlin)", "iOS (Swift)"],
    }
}

print(f"Actual:   {actual_skills}")
print(f"Expected: {expected_skills}")
assert actual_skills == expected_skills

The failing test in the preceding cell, though jarring, is a good thing! It helped us catch a bug: There was some information loss as some skills were not included in the result. Let's try to get this test to pass with better prompting

In [ ]:
import json

template = """
[INST] <<SYS>>
You are an assistant responsible for extracting attributes from unstructured text and returning them as a valid JSON. 
In your response, DO NOT include any text other than the JSON response.
Keys and values should be quoted with ".

Example: This is a resume, and I'm listing my technical skills:
- Fruits: Apple, Banana, Coconut
- Raw Vegetables: Lettuce, Cabbage, Zucchini, Carrots (Diced, Sliced)

The response should be:
{{"technical_skills": {{"fruits": ["Apple", "Banana", "Coconut"], "raw_vegetables": ["Lettuce", "Cabbage", "Zucchini", "Carrots (Diced, Sliced)"]}}}}

<</SYS>>

Extract technical skills from the given document using only information in the following document: {resume}
Only include the JSON response.

[/INST]
"""
prompt_template = PromptTemplate(
    input_variables=["resume"],
    template=template,
)
prompt = prompt_template.format(resume=RESUME)
result = llm(prompt)

print(result)

In [ ]:
actual_skills = json.loads(result)
expected_skills = {
    "technical_skills": {
        "languages": ["Java", "Python", "JavaScript", "C#"],
        "web": ["HTML5", "CSS3", "Bootstrap", "React", "Angular", "Node.js"],
        "mobile": ["Android (Java, Kotlin)", "iOS (Swift)"],
    }
}

print(f"Actual:   {actual_skills}")
print(f"Expected: {expected_skills}")
assert actual_skills == expected_skills

## Exercise 3: Using an LLM to evaluate itself (or another LLM)

In [ ]:
template = """
[INST]
<<SYS>>
You are a helpful assistant, skilled at providing succinct and accurate summaries of an applicant based on their resume. In your response, include only the summary without any preamble.
<</SYS>>

Instruction: Generate a two-sentence summary of: {resume}
[/INST]
"""
summarise_resume_prompt_template = PromptTemplate(
    input_variables=["resume"],
    template=template,
)
summarise_resume_prompt = summarise_resume_prompt_template.format(resume=RESUME)

summary = llm(summarise_resume_prompt)
print(summary)

In [ ]:
evaluation_template = """
<SYS>
You are a strict evaluator responsible for checking if summaries are accurate or not.
</SYS>
[INST]
<Resume>
{resume}
</Resume>

<Summary>
{summary}
</Summary>

Instruction: Evaluate if <Summary> is an accurate summary of <Resume>. Present response in a JSON format with keys of score and a short reason.
[/INST]
"""
evaluation_prompt_template = PromptTemplate(
    input_variables=["summary", "resume"],
    template=evaluation_template,
)
evaluation_prompt = evaluation_prompt_template.format(summary=summary, resume=RESUME)

# print(llm(evaluation_prompt))

In [ ]:
RESUME_2 = """
Jane Doe, Marketing Manager

Professional Experience:

Marketing Manager | Horizon Marketing Solutions | 2018 - Present
- Spearheaded end-to-end marketing efforts for a diverse portfolio of clients, resulting in a 20 percent average increase in annual revenue.
- Led a cross-functional team of 10 professionals, fostering a collaborative environment and achieving a 30 percent improvement in campaign efficiency.
- Developed and executed data-driven marketing strategies, resulting in a 25 percent boost in online conversions and a 15 percent increase in social media engagement.
- Conducted in-depth market analysis, competitor assessments, and consumer behavior studies to identify new opportunities and optimize existing campaigns.
- Collaborated closely with clients to align marketing strategies with their business goals, resulting in a 95 percent client retention rate.

Skills:
- Marketing Strategy, Brand Development, Campaign Management, Team Leadership, Market Research, Digital Marketing, Data Analysis, Cross-functional Collaboration

"""

summarise_resume_prompt_2 = summarise_resume_prompt_template.format(resume=RESUME_2)
summary_2 = llm(summarise_resume_prompt_2)
print(summary_2)

In [ ]:
evaluation_prompt_2 = evaluation_prompt_template.format(summary=summary_2, resume=RESUME_2)

print(llm(evaluation_prompt_2))

In [ ]:
evaluation_prompt_3 = evaluation_prompt_template.format(summary="Bob Dole is a zookeeper", resume=RESUME_2)

print(llm(evaluation_prompt_3))